In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

Leemos los datos descargados con ayuda del API de idealista, al costado de cada csv se indica el mes de descarga de cada Dataframe.

In [4]:
path='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'
df_12_2024=pd.read_csv(path+"idealista_12_2024.csv")
df_01_2025_01=pd.read_csv(path+"idealista_sale_01_01_2025_1.csv")
df_01_2025_02=pd.read_csv(path+"idealista_sale_01_01_2025_2.csv")
df_01_2025_03=pd.read_csv(path+"idealista_sale_02_2025_1.csv")
df_01_2025_04=pd.read_csv(path+"idealista_sale_02_2025_2.csv")
"""df_03=pd.read_csv(path+"idealista_03.csv")
df_04=pd.read_csv(path+"idealista_04.csv")
df_05=pd.read_csv(path+"idealista_05.csv")"""


'df_03=pd.read_csv(path+"idealista_03.csv")\ndf_04=pd.read_csv(path+"idealista_04.csv")\ndf_05=pd.read_csv(path+"idealista_05.csv")'

Agregamos una columna de mes a cada DataFrame y eliminamos la columna 'priceInfo'

In [5]:
"""df_02['Month']='Febrero'
df_03['Month']='Marzo'
df_04['Month']='Abril'
df_05['Month']='Mayo'"""

df_12_2024['Month']='Diciembre'
df_01_2025_01['Month']='Enero'
df_01_2025_02['Month']='Enero'
df_01_2025_03['Month']='Febrero'
df_01_2025_04['Month']='Febrero'

columnas=['index', 'propertyCode', 'thumbnail', 'externalReference', 'numPhotos',
       'floor', 'price', 'propertyType', 'operation', 'size', 'exterior',
       'rooms', 'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'distance', 'description', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'detailedType',
       'suggestedTexts', 'hasPlan', 'has3DTour', 'has360', 'hasStaging',
       'topNewDevelopment', 'topPlus', 'parkingSpace',
       'newDevelopmentFinished', 'highlight', 'Month']


df_12_2024=df_12_2024[columnas]
df_01_2025_01=df_01_2025_01[columnas]
df_01_2025_02=df_01_2025_02[columnas]
"""df_02=df_02[columnas]
df_03=df_03[columnas]
df_04=df_04[columnas]
df_05=df_05[columnas]"""

'df_02=df_02[columnas]\ndf_03=df_03[columnas]\ndf_04=df_04[columnas]\ndf_05=df_05[columnas]'

Verificamos que cada Dataframe tiene la misma estructura    

In [6]:
print(df_12_2024.shape)
print(df_01_2025_01.shape)
print(df_01_2025_02.shape)
print(df_01_2025_03.shape)
print(df_01_2025_04.shape)
"""print(df_02.shape)
print(df_03.shape)
print(df_04.shape)
print(df_05.shape)"""

(4850, 42)
(5000, 42)
(2350, 42)
(4950, 43)
(2250, 43)


'print(df_02.shape)\nprint(df_03.shape)\nprint(df_04.shape)\nprint(df_05.shape)'

Unimos los Dataframes y eliminanos los duplicados en función de la columna 'propertyCode'manteniendo la última oferta de idealista 

In [7]:
# Unir los DataFrames en una lista
# dfs = [df_02,df_03,df_04,df_05]
dfs = [df_12_2024,df_01_2025_01,df_01_2025_02,df_01_2025_03,df_01_2025_04]

# Concatenar los DataFrames
merged_df = pd.concat(dfs,ignore_index=True)

# Eliminar filas duplicadas basadas en la columna 'propertyCode', manteniendo la última aparición
merged_df = merged_df.drop_duplicates(subset='propertyCode', keep='last')

merged_df.shape

(11070, 43)

Verificamos que no existan duplicados en merged_df

In [8]:
# Verificar si hay valores duplicados en la columna 'propertyCode'
duplicated_values = merged_df['propertyCode'].duplicated(keep=False)

# Mostrar las filas que tienen valores duplicados
duplicated_rows = merged_df[duplicated_values]

# Contar cuántas veces se repite cada valor duplicado
duplicate_counts = merged_df['propertyCode'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]

# Mostrar resultados
print("Filas con valores duplicados en 'propertyCode':")
print(duplicated_rows)

print("\nConteo de valores duplicados en 'propertyCode':")
print(duplicate_counts)

Filas con valores duplicados en 'propertyCode':
Empty DataFrame
Columns: [index, propertyCode, thumbnail, externalReference, numPhotos, floor, price, propertyType, operation, size, exterior, rooms, bathrooms, address, province, municipality, district, country, neighborhood, latitude, longitude, showAddress, url, distance, description, hasVideo, status, newDevelopment, hasLift, priceByArea, detailedType, suggestedTexts, hasPlan, has3DTour, has360, hasStaging, topNewDevelopment, topPlus, parkingSpace, newDevelopmentFinished, highlight, Month, priceInfo]
Index: []

[0 rows x 43 columns]

Conteo de valores duplicados en 'propertyCode':
Series([], Name: count, dtype: int64)


Hacemos un groub by para ver la distribución de las propiedades por municipio

In [9]:
merged_df.groupby('municipality').count()['province']

municipality
Alcobendas                   34
Alcorcón                    191
Boadilla del Monte           16
Coslada                     106
Fuenlabrada                   1
Getafe                      341
La Moraleja                  24
Las Rozas de Madrid           3
Leganés                     390
Madrid                     9714
Majadahonda                  22
Pozuelo de Alarcón          165
Rivas-Vaciamadrid            24
San Fernando de Henares      39
Name: province, dtype: int64

Filtramos el Dataframe para tener solo las propiedades dentro de la municipalidad de Madrid, y las variables a trabajar. Finalmente eliminamos cualquier oferta que tenga un NaN.

In [10]:
filtered_df=merged_df[merged_df['municipality']=='Madrid']

variables=['propertyCode','price', 'floor', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms','district','neighborhood','latitude', 'longitude', 'distance',
           'status', 'newDevelopment', 'hasLift','priceByArea', 'parkingSpace', 'newDevelopmentFinished','Month','province']

filtered_df=filtered_df[variables]

filtered_df


,propertyCode,price,floor,propertyType,size,exterior,rooms,bathrooms,district,neighborhood,...,longitude,distance,status,newDevelopment,hasLift,priceByArea,parkingSpace,newDevelopmentFinished,Month,province
50,106398551,195000.0,NaN,flat,51.0,NaN,2,2,Carabanchel,San Isidro,...,-3.733760,3855,good,False,False,3824.0,NaN,NaN,Diciembre,Madrid
52,105507380,69500.0,NaN,flat,25.0,NaN,1,1,Puente de Vallecas,Numancia,...,-3.659877,4569,good,False,False,2780.0,NaN,NaN,Diciembre,Madrid
54,106012042,480000.0,5,flat,80.0,True,3,2,Tetuán,Bellas Vistas,...,-3.701680,4182,good,False,True,6000.0,NaN,NaN,Diciembre,Madrid
60,96552179,104000.0,3,flat,63.0,NaN,3,1,Carabanchel,Puerta Bonita,...,-3.741242,5556,good,False,False,1651.0,NaN,NaN,Diciembre,Madrid
64,104913355,83000.0,NaN,flat,103.0,NaN,3,1,Usera,12 de Octubre-Orcasur,...,-3.702746,5274,good,False,False,806.0,NaN,NaN,Diciembre,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,106906036,186000.0,2,flat,95.0,True,3,1,Puente de Vallecas,Entrevías,...,-3.672079,4432,good,False,False,1958.0,NaN,NaN,Febrero,Madrid
19395,84170210,600000.0,10,flat,60.0,True,2,1,Tetuán,Cuatro Caminos,...,-3.693345,4021,good,False,True,10000.0,NaN,NaN,Febrero,Madrid
19396,106322695,385000.0,5,penthouse,100.0,True,2,1,Tetuán,Cuzco-Castillejos,...,-3.691216,4842,good,False,True,3850.0,NaN,NaN,Febrero,Madrid
19397,107206902,299000.0,2,flat,40.0,True,2,1,Centro,Chueca-Justicia,...,-3.699803,939,good,False,True,7475.0,NaN,NaN,Febrero,Madrid


In [11]:
# Vemos los tipos de datos para cada columna si es númerico, bool u object
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9714 entries, 50 to 19399
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            9714 non-null   int64  
 1   price                   9714 non-null   float64
 2   floor                   9078 non-null   object 
 3   propertyType            9714 non-null   object 
 4   size                    9714 non-null   float64
 5   exterior                9090 non-null   object 
 6   rooms                   9714 non-null   int64  
 7   bathrooms               9714 non-null   int64  
 8   district                9714 non-null   object 
 9   neighborhood            9714 non-null   object 
 10  latitude                9714 non-null   float64
 11  longitude               9714 non-null   float64
 12  distance                9714 non-null   int64  
 13  status                  9535 non-null   object 
 14  newDevelopment          9714 non-null   boo

In [12]:
filtered_df['floor'].unique()

array([nan, '5', '3', '1', 'bj', 'ss', 'st', '2', '4', '6', 'en', '7',
       '14', '8', '9', '10', '12', '11', '-1', '15', '16', '13', '17',
       '21', '-2'], dtype=object)

Corregimos los datos de la columna floor

In [13]:
# Reemplazar 'bj' y 'en' por '0' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace(['bj','en'], 0)

# Reemplazar 'ss' por '-1' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace('ss', '-1')

# Eliminamos las 'st' ya que son sobretecho y no sabes cual es el valor de la última planta de cada piso
filtered_df = filtered_df.drop(filtered_df[filtered_df['floor'] == 'st'].index)

# Cambiar los valores de 60 a 6 en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace('60', '6')

# Convertir la columna 'floor' de tipo 'object' a tipo 'integer', manteniendo NaNs
filtered_df['floor'] = pd.to_numeric(filtered_df['floor'], errors='coerce')

# Calcular la mediana de 'floor' para cada 'propertyType'
median_floors = filtered_df.groupby('propertyType')['floor'].median()

# Definir una función para rellenar NaNs con la mediana correspondiente
def fill_floor(row):
    if pd.isna(row['floor']):
        if row['propertyType'] == 'chalet':
            return 0
        else:
            return median_floors[row['propertyType']]
    else:
        return row['floor']

# Aplicar la función para rellenar NaNs
filtered_df['floor'] = filtered_df.apply(fill_floor, axis=1)

# Convertir la columna 'floor' a tipo 'int'
filtered_df['floor'] = filtered_df['floor'].astype(int)

filtered_df['floor'].unique()


array([ 2,  5,  3,  0,  1, -1,  4,  6,  7, 14,  8,  9, 10, 12, 11, 15, 16,
       13, 17, 21, -2])

Corregimos los datos de la columna exterior

In [14]:
# Reemplazar 'Nan' por 'False' en la columna 'exterior'
filtered_df.fillna({'exterior': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('False', 0)

# Convertir la columna 'exterior' de tipo 'object' a tipo 'integer'
filtered_df['exterior'] = filtered_df['exterior'].astype(int)

# Mostrar el resultado
filtered_df['exterior'].unique()

array([0, 1])

Corregimos los datos de la columna rooms

In [15]:
# Incrementar en una la cantidad de habitaciones en los studios que tienen 0 habitaciones.
filtered_df['rooms'] = filtered_df.apply(lambda row: row['rooms'] + 1 if row['propertyType'] == 'studio' else row['rooms'], axis=1)

# Eliminar filas donde 'rooms' o 'bathrooms' es 0
filtered_df = filtered_df.drop(filtered_df[(filtered_df['rooms'] == 0) | (filtered_df['bathrooms'] == 0)].index)

filtered_df['rooms'].unique()

array([ 2,  1,  3,  4,  6,  5,  7,  8,  9, 14, 13], dtype=int64)

Corregimos los datos de la columna newDevelopment

In [16]:
# Reemplazar 'True' por '1' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('False', 0)

# Convertir la columna 'newDevelopment' de tipo 'object' a tipo 'integer'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopment'].unique()

array([0, 1])

Corregimos los datos de la columna Status


In [17]:
# Si en la columna 'newDevelopment' el valor es 1, en la columna Status remplazar por newdevelopment
filtered_df.loc[(filtered_df['newDevelopment'] == 1) & (filtered_df['status'].isna()), 'status'] = 'newdevelopment'

print(filtered_df['status'].unique())

['good' 'renew' 'newdevelopment']


Corregimos los datos de la columna hasLift

In [18]:
# Reemplazar 'Nan' por 'False' en la columna 'hasLift'
filtered_df.fillna({'hasLift': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('False', 0)

# Convertir la columna 'hasLift' de tipo 'object' a tipo 'integer'
filtered_df['hasLift'] = filtered_df['hasLift'].astype(int)

# Mostrar el resultado
filtered_df['hasLift'].unique()

array([0, 1])

In [19]:
# Reemplazar 'Nan' por 'False' en la columna 'newDevelopmentFinished'
filtered_df.fillna({'newDevelopmentFinished': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('False', 0)

# Convertir la columna 'newDevelopmentFinished' de tipo 'object' a tipo 'integer'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopmentFinished'].unique()

array([0, 1])

Corregimos los datos de la columna parkingSpace
y generamos tres columnas : hasParkingSpace , isParkingSpaceIncludedInPrice , parkingSpacePrice

In [20]:
import ast

# Función para procesar cada fila
def process_row(row):
    if pd.isna(row):
        return pd.Series({'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 0})
    else:
        try:
            row_dict = ast.literal_eval(row)  # Convertir la cadena a un diccionario
            has_parking_space = row_dict.get('hasParkingSpace', False)
            is_included = row_dict.get('isParkingSpaceIncludedInPrice', False)
            price = row_dict.get('parkingSpacePrice', 0)
        except (SyntaxError, ValueError):  # Manejar el caso de que la cadena no sea un diccionario válido
            has_parking_space = False
            is_included = False
            price = 0
        return pd.Series({'hasParkingSpace': has_parking_space, 'isParkingSpaceIncludedInPrice': is_included, 'parkingSpacePrice': price})

# Aplicar la función a cada fila
new_columns = filtered_df['parkingSpace'].apply(process_row)

# Concatenar los resultados al DataFrame original
filtered_df = pd.concat([filtered_df, new_columns], axis=1)

# Eliminar la columna 'parkingSpace' tras generar las tres columnas
filtered_df = filtered_df.drop(columns=['parkingSpace'])

Corregimos los datos de la columna hasParkingSpace

In [21]:
# Reemplazar 'Nan' por 'False' en la columna 'hasParkingSpace'
filtered_df.fillna({'hasParkingSpace': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('False', 0)

# Convertir la columna 'hasParkingSpace' de tipo 'object' a tipo 'integer' 
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].astype(int)

# Mostrar el resultado
filtered_df['hasParkingSpace'].unique()

array([0, 1])

Corregimos los datos de la columna isParkingSpaceIncludedInPrice

In [22]:
# Reemplazar 'Nan' por 'False' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df.fillna({'isParkingSpaceIncludedInPrice': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('False', 0)

# Convertir la columna 'isParkingSpaceIncludedInPrice' de tipo 'object' a tipo 'integer' 
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].astype(int)

# Mostrar el resultado
filtered_df['isParkingSpaceIncludedInPrice'].unique()

array([0, 1])

Media del precio m2 por distrito

In [23]:
# Calcular la media de priceByArea por distrito
mean_price_by_district = filtered_df[['district', 'priceByArea']].groupby('district').mean()

# Crear un diccionario a partir de la serie resultante
mean_price_dict = mean_price_by_district['priceByArea'].to_dict()

# Crear la nueva columna priceByAreaByDistrict en el DataFrame original
filtered_df['priceByAreaByDistrict'] = filtered_df['district'].map(mean_price_dict)

# Mostrar los primeros resultados para verificar
filtered_df

,propertyCode,price,floor,propertyType,size,exterior,rooms,bathrooms,district,neighborhood,...,newDevelopment,hasLift,priceByArea,newDevelopmentFinished,Month,province,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,priceByAreaByDistrict
50,106398551,195000.0,2,flat,51.0,0,2,2,Carabanchel,San Isidro,...,0,0,3824.0,0,Diciembre,Madrid,0,0,0.0,3159.709202
52,105507380,69500.0,2,flat,25.0,0,1,1,Puente de Vallecas,Numancia,...,0,0,2780.0,0,Diciembre,Madrid,0,0,0.0,2770.656012
54,106012042,480000.0,5,flat,80.0,1,3,2,Tetuán,Bellas Vistas,...,0,1,6000.0,0,Diciembre,Madrid,0,0,0.0,4999.250000
60,96552179,104000.0,3,flat,63.0,0,3,1,Carabanchel,Puerta Bonita,...,0,0,1651.0,0,Diciembre,Madrid,0,0,0.0,3159.709202
64,104913355,83000.0,2,flat,103.0,0,3,1,Usera,12 de Octubre-Orcasur,...,0,0,806.0,0,Diciembre,Madrid,0,0,0.0,3086.827068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,106906036,186000.0,2,flat,95.0,1,3,1,Puente de Vallecas,Entrevías,...,0,0,1958.0,0,Febrero,Madrid,0,0,0.0,2770.656012
19395,84170210,600000.0,10,flat,60.0,1,2,1,Tetuán,Cuatro Caminos,...,0,1,10000.0,0,Febrero,Madrid,0,0,0.0,4999.250000
19396,106322695,385000.0,5,penthouse,100.0,1,2,1,Tetuán,Cuzco-Castillejos,...,0,1,3850.0,0,Febrero,Madrid,0,0,0.0,4999.250000
19397,107206902,299000.0,2,flat,40.0,1,2,1,Centro,Chueca-Justicia,...,0,1,7475.0,0,Febrero,Madrid,0,0,0.0,6548.581737


Media del precio m2 por barrio

In [24]:
# Calcular la media de priceByArea por distrito
mean_price_by_neighborhood = filtered_df[['neighborhood', 'priceByArea']].groupby('neighborhood').mean()

# Crear un diccionario a partir de la serie resultante
mean_price_dict = mean_price_by_neighborhood['priceByArea'].to_dict()

# Crear la nueva columna priceByAreaByneighborhood en el DataFrame original
filtered_df['priceByAreaByneighborhood'] = filtered_df['neighborhood'].map(mean_price_dict)

# Mostrar los primeros resultados para verificar
filtered_df

,propertyCode,price,floor,propertyType,size,exterior,rooms,bathrooms,district,neighborhood,...,hasLift,priceByArea,newDevelopmentFinished,Month,province,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,priceByAreaByDistrict,priceByAreaByneighborhood
50,106398551,195000.0,2,flat,51.0,0,2,2,Carabanchel,San Isidro,...,0,3824.0,0,Diciembre,Madrid,0,0,0.0,3159.709202,3419.876033
52,105507380,69500.0,2,flat,25.0,0,1,1,Puente de Vallecas,Numancia,...,0,2780.0,0,Diciembre,Madrid,0,0,0.0,2770.656012,2959.143713
54,106012042,480000.0,5,flat,80.0,1,3,2,Tetuán,Bellas Vistas,...,1,6000.0,0,Diciembre,Madrid,0,0,0.0,4999.250000,4753.000000
60,96552179,104000.0,3,flat,63.0,0,3,1,Carabanchel,Puerta Bonita,...,0,1651.0,0,Diciembre,Madrid,0,0,0.0,3159.709202,2880.558333
64,104913355,83000.0,2,flat,103.0,0,3,1,Usera,12 de Octubre-Orcasur,...,0,806.0,0,Diciembre,Madrid,0,0,0.0,3086.827068,3226.137931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,106906036,186000.0,2,flat,95.0,1,3,1,Puente de Vallecas,Entrevías,...,0,1958.0,0,Febrero,Madrid,0,0,0.0,2770.656012,2032.156627
19395,84170210,600000.0,10,flat,60.0,1,2,1,Tetuán,Cuatro Caminos,...,1,10000.0,0,Febrero,Madrid,0,0,0.0,4999.250000,5973.631148
19396,106322695,385000.0,5,penthouse,100.0,1,2,1,Tetuán,Cuzco-Castillejos,...,1,3850.0,0,Febrero,Madrid,0,0,0.0,4999.250000,6185.813559
19397,107206902,299000.0,2,flat,40.0,1,2,1,Centro,Chueca-Justicia,...,1,7475.0,0,Febrero,Madrid,0,0,0.0,6548.581737,7762.787402


Verificamos que los datos númericos y categoricos estén correctamente.

In [25]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9616 entries, 50 to 19399
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   propertyCode                   9616 non-null   int64  
 1   price                          9616 non-null   float64
 2   floor                          9616 non-null   int32  
 3   propertyType                   9616 non-null   object 
 4   size                           9616 non-null   float64
 5   exterior                       9616 non-null   int32  
 6   rooms                          9616 non-null   int64  
 7   bathrooms                      9616 non-null   int64  
 8   district                       9616 non-null   object 
 9   neighborhood                   9616 non-null   object 
 10  latitude                       9616 non-null   float64
 11  longitude                      9616 non-null   float64
 12  distance                       9616 non-null   int6

Guardamos el Dataframe procesado en un archivo csv

In [ ]:
#Definimos el nombre del archivo
nombre_cvs='datos_procesados_01_2025.csv'

# Definimos la ruta
ruta='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'

# Ruta donde guardar el archivo CSV
ruta_csv = ruta+nombre_cvs

# Guardar el DataFrame en un archivo CSV
filtered_df.to_csv(ruta_csv, index=False)

: 